 # 教師データ(画像とlst形式のメタデータ)を増幅させて、RecodIOファイルに変換

## 必要なモジュールをインストール

In [ ]:
!pip install imgaug tqdm

## 複数の教師データをマージする

複数のlstファイルと画像ファイルをマージする。
その際に、画像は指定したサイズの正方形へとリサイズする。
lstファイルは[mxnetの物体検出用のフォーマット](https://mxnet.incubator.apache.org/api/python/image/image.html)（ヘッダーサイズが2で一つのラベルデータの数は5、エクストラヘッダーは無し）を想定。


In [ ]:
# lstファイルのパス
lst_path_list = ['path/to/lst1', 'path/to/lst2']

# 画像ファイルの位置(順序はlstファイルと対応づける)
img_root_path_list = ['path/to/lst1', 'path/to/lst2']
    

# 読み込んだlstファイルをマージしたもの出力先ルート
merged_root_path = './data/merged'

# 画像サイズ(変換後の画像サイズ: img_edge_size * img_edge_size)
img_edge_size = 512

### 関数定義


In [ ]:
def create_lst(file_path, index, label_data):
    """
    lst形式のデータ（文字列）を作成
    """
    header_size = 2
    label_width = 5
    return '\t'.join([
        str(index),
        str(header_size),
        str(label_width),
        '\t'.join(label_data),
        file_path])

def read_lst(dat):
    """
    lst形式のデータ(文字列)の内容を読み込む
    """
    dat_list = dat.split('\t')
    index = int(dat_list[0])
    
    header_size = int(dat_list[1])
    assert header_size == 2, 'header_sizeは２を想定:'+str(header_size)
    
    label_width = int(dat_list[2])
    assert label_width == 5, 'label_widthは5を想定: '+str(label_width)
    
    label_data = dat_list[3:-1]
    assert (len(label_data) % label_width) == 0 , 'label_dataの長さはlabel_widthの倍数のはず : '
    
    file_path = dat_list[-1]
    
    return (index, header_size, label_width, label_data, file_path)
    


### 処理

In [ ]:
import os
from os import path
import shutil
from PIL import Image
from tqdm import tqdm

assert len(lst_path_list) == len(img_root_path_list), "lst_path_listとimg_root_path_listの長さは同じのはず"


#マージしたlstファイルと画像ファイルの出力先
output_lst_path = path.join(merged_root_path, "lst.lst")
output_img_root_path = path.join(merged_root_path, "img")

# 出力先をリセット
if path.isdir(merged_root_path):
    shutil.rmtree(merged_root_path)
    
os.makedirs(merged_root_path)
os.makedirs(output_img_root_path)

# マージ処理開始
merged_lst = []
for lst_path, img_root_path in tqdm(zip(lst_path_list, img_root_path_list)):
    with open(lst_path) as lst_f:
        for line in tqdm(lst_f.readlines()):
            line = line.strip()
            if not line: continue
            
            #lst形式のデータを読み取って、変数に入れる
            index, header_size, label_width, label_data, img_path = read_lst(line)
            img_path = path.join(img_root_path, img_path)
            
            merged_index = len(merged_lst) + 1
            
            
            # 画像ファイル名をcountに書き換える
            after_img_name =  str(merged_index) + path.splitext(img_path)[1]
            after_img_path = path.join(output_img_root_path, after_img_name)
            
            #マージ後ファイル出力先へ画像をコピー
            img = Image.open(img_path)
            
            # 余白は黒(0,0,0)にして正方形の画像に変換し、その後指定したサイズへ変換
            img.thumbnail((img_edge_size, img_edge_size))
            img.save(after_img_path)
            
            
            #lst形式のテキストを作成
            lst_dat = create_lst(after_img_name, merged_index, label_data)
            merged_lst.append(lst_dat)

            
# 作成したデータを要素ごとに改行して書き出す
with open(output_lst_path, 'w') as out_f:
    out_f.write('\n'.join(merged_lst))
        

# 教師データを増幅

データを検証用と学習用に分割し、それぞれのデータを[imgaug](https://github.com/aleju/imgaug)を使って増幅させる。
処理終了後、検証用と学習用それぞれのデータ数を表示。  

In [ ]:
# 検証用データの割合 0の場合は学習データのみ作成
validate_ratio = 0.2

#読み込むlstファイル
lst_path = output_lst_path
img_root_path = output_img_root_path

# 読み込んだlstファイルをマージしたもの出力先ルート
augmented_root_path = './data/augmented'



### 画像増幅処理の定義
`augs`に定義された処理が実行されます。  
必要に応じて`augs`や`aug_templates`を変更する。  

In [ ]:
import numpy as np
import math
from PIL import Image
from scipy import misc
import imgaug as ia
from imgaug import augmenters as iaa
from matplotlib import pyplot as plt

# シードを固定
ia.seed(1)

# 画像増幅のためのaugmentorを定義(必要に応じて変える)
aug_templates = [
    iaa.Invert(1, per_channel=0.5), #  各ピクセルの値を反転させる
    iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.25)), #ところどころ欠落させる
    iaa.CoarseDropout((0.03, 0.15), size_percent=0.02, per_channel=0.8), # ところどころ色を変える
    iaa.CoarseSaltAndPepper(0.2, size_percent=(0.05, 0.1)), # 白と黒のノイズ
    iaa.WithChannels(0, iaa.Affine(rotate=(0,10))), # 赤い値を傾ける
    iaa.FrequencyNoiseAlpha( # 決まった形のノイズを加える
        first=iaa.EdgeDetect(1),
        per_channel=0.5
    ),
    iaa.ElasticTransformation(sigma=0.5, alpha=1.0), # モザイクをかける
    iaa.AddToHueAndSaturation(value=25), # 色調と彩度に値を追加
    iaa.Emboss(alpha=1.0, strength=1.5), # 浮き出し加工
    iaa.Superpixels(n_segments=100, p_replace=0.5), # superpixel表現にして、各セル内を一定確率でセルの平均値で上書きする
    iaa.Fliplr(1.0),
    iaa.Flipud(1.0)
]


# 実行する画像増幅処理一覧(必要に応じて変える)
augs = [
    iaa.Noop(), # 無変換
    iaa.SomeOf(1, aug_templates),
    iaa.SomeOf(1, aug_templates),
    iaa.SomeOf(1, aug_templates),
    iaa.SomeOf(2, aug_templates),
    iaa.SomeOf(2, aug_templates),
    iaa.SomeOf(2, aug_templates),
    iaa.SomeOf(3, aug_templates),
]



### 処理定義

In [ ]:
import random
import copy

assert validate_ratio < 1.0, "validate_ratio は1以下のはず" + str(validate_ratio)

#マージしたlstファイルと画像ファイルの出力先
train_augmented_lst_path = path.join(augmented_root_path, "train.lst")
train_augmented_img_root_path = path.join(augmented_root_path, "train")

val_augmented_lst_path = path.join(augmented_root_path, "val.lst")
val_augmented_img_root_path = path.join(augmented_root_path, "val")



# 出力先をリセット
if path.isdir(augmented_root_path):
    shutil.rmtree(augmented_root_path)

os.makedirs(augmented_root_path)
os.makedirs(train_augmented_img_root_path)
os.makedirs(val_augmented_img_root_path)

train_augmented_lst = []
val_augmented_lst = []
with open(lst_path) as lst_f:
    for line in tqdm(lst_f.readlines()):
            line = line.strip()
            if not line: continue
            
            #lst形式のデータを読み取って、変数に入れる
            origin_img_index, header_size, label_width, label_data, img_path = read_lst(line)
            img_path = path.join(img_root_path, img_path)
            
            # 画像を読み込む
            target_img = np.array(Image.open(img_path))
            
            # バウンディングボックスを生成
            img_height = target_img.shape[0]
            img_width = target_img.shape[1]
            bbs = []
            for bb_index in range(len(label_data)//label_width):
                bbs.append(ia.BoundingBox(
                    x1 = float(label_data[bb_index * label_width + 1]) * img_width,
                    y1 = float(label_data[bb_index * label_width + 2]) * img_height,
                    x2 = float(label_data[bb_index * label_width + 3]) * img_width,
                    y2 = float(label_data[bb_index * label_width + 4]) * img_height
                ))
            bbs_on_img = ia.BoundingBoxesOnImage(bbs, shape = target_img.shape)
            
            # 指定した確率で検証用データとして割り当てる
            if random.random() < validate_ratio:
                augmented_lst = val_augmented_lst
                augmented_img_root_path = val_augmented_img_root_path
            else:
                augmented_lst = train_augmented_lst
                augmented_img_root_path = train_augmented_img_root_path

            
            #画像
            aug_num = len(augs)
            for aug_index, aug in enumerate(augs):
                
                #画像増幅する
                aug_img = aug.augment_image(target_img)
                aug_bbs = aug.augment_bounding_boxes([bbs_on_img])[0]
                
                
                image_index = len(augmented_lst) + 1
                
                # 増幅した画像ファイル名
                after_img_name = "{0:05d}_{1:03d}{2}".format(origin_img_index, aug_index+1, path.splitext(img_path)[1])
                after_img_path = path.join(augmented_img_root_path, after_img_name)
                
                # 増幅した画像を保存
                Image.fromarray(aug_img).save(after_img_path)
                
                # ラベルデータを上書き
                aug_label_data = copy.deepcopy(label_data)
                for bb_index in range(len(label_data)//label_width):
                    aug_label_data[bb_index * label_width + 1] = str(aug_bbs.bounding_boxes[bb_index].x1 /  img_width)
                    aug_label_data[bb_index * label_width + 2] = str(aug_bbs.bounding_boxes[bb_index].y1 /  img_height)
                    aug_label_data[bb_index * label_width + 3] = str(aug_bbs.bounding_boxes[bb_index].x2 /  img_width)
                    aug_label_data[bb_index * label_width + 4] = str(aug_bbs.bounding_boxes[bb_index].y2 /  img_height)


                # 増幅画像用のlst形式のテキストを作成
                lst_dat = create_lst(after_img_name, image_index, aug_label_data)

                augmented_lst.append(lst_dat)
            

            
# 作成したデータを要素ごとに改行して書き出す
with open(train_augmented_lst_path, 'w') as out_f:
    out_f.write('\n'.join(train_augmented_lst))
    
if len(val_augmented_lst) > 0:
    with open(val_augmented_lst_path, 'w') as out_f:
        out_f.write('\n'.join(val_augmented_lst))

print("train data: ",len(train_augmented_lst))
print("validation data: ", len(val_augmented_lst))
            

# RecordIO形式に変換

In [ ]:
import os
import urllib.request

def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

    
    
# Tool for creating lst file
download('https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py')

In [ ]:
!python im2rec.py ./data/augmented/train.lst ./data/augmented/train/ --num-thread 4 --pack-label 
!python im2rec.py ./data/augmented/val.lst ./data/augmented/val/ --num-thread 4 --pack-label 